Implementing Grid Search on keras, based on this <a href="https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/"> tutorial</a> 

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("test.csv")
df = df.dropna()

In [3]:
df["ha"] = np.where(df["ha"].eq("h"),1,0)

In [4]:
X = df[['opp_goal_difference_last', 'ha', 'opp_goal_difference_5MA', 'opp_goal_difference_10MA',
       'since_last_win', 'goal_difference_5MA', 'goal_difference_10MA', 'goal_difference_last',
        'yellow_cards_referee_hist','odds_5MA', 'odds_10MA', 'odds_last'
       ]].values
y = df["class_var"]

In [5]:
n_samples = len(y)
n_classes = len(y.unique())
class_weights = n_samples / (n_classes * y.value_counts())
class_weights = class_weights.to_dict()

We split the dataset into train and test. We shuffle the examples as the order is not important here.

In [6]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)

We scale with the Standard Scaler. That is the features are normalized. Models usually work better when this is the case.

In [7]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, stratify=y)

model = RandomForestClassifier(max_depth=5, random_state=0, class_weight=class_weights)

model.fit(X_train, y_train)

RandomForestClassifier(class_weight={0: 1.286878565607172,
                                     1: 0.8997150997150997,
                                     2: 0.8997150997150997},
                       max_depth=5, random_state=0)

In [9]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.28      0.32      0.30       164
           1       0.57      0.48      0.52       234
           2       0.53      0.56      0.54       234

    accuracy                           0.47       632
   macro avg       0.46      0.45      0.45       632
weighted avg       0.48      0.47      0.47       632



In [20]:
class_weights_grid = []
for i in range(6):
    c = class_weights.copy()
    c.update({0:i})
    class_weights_grid.append(c)

In [21]:
param_grid = { 
    'class_weight': class_weights_grid,
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}
grid = GridSearchCV(estimator=model, param_grid=param_grid, verbose=3)
grid_result = grid.fit(X, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 5 folds for each of 360 candidates, totalling 1800 fits
[CV 1/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 0}, criterion=gini, max_depth=4, max_features=auto, n_estimators=200;, score=0.524 total time=   0.4s
[CV 2/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 0}, criterion=gini, max_depth=4, max_features=auto, n_estimators=200;, score=0.492 total time=   0.3s
[CV 3/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 0}, criterion=gini, max_depth=4, max_features=auto, n_estimators=200;, score=0.505 total time=   0.3s
[CV 4/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 0}, criterion=gini, max_depth=4, max_features=auto, n_estimators=200;, score=0.506 total time=   0.3s
[CV 5/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 0}, criterion=gini, max_depth=4, max_features=auto, n_estimators=200;, score=0.480 total time=   0.3s
[CV 1/5] END class_weight={2: 0.89971509971509

[CV 2/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 0}, criterion=gini, max_depth=5, max_features=sqrt, n_estimators=500;, score=0.492 total time=   1.0s
[CV 3/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 0}, criterion=gini, max_depth=5, max_features=sqrt, n_estimators=500;, score=0.505 total time=   1.0s
[CV 4/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 0}, criterion=gini, max_depth=5, max_features=sqrt, n_estimators=500;, score=0.515 total time=   1.0s
[CV 5/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 0}, criterion=gini, max_depth=5, max_features=sqrt, n_estimators=500;, score=0.480 total time=   0.9s
[CV 1/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 0}, criterion=gini, max_depth=5, max_features=log2, n_estimators=200;, score=0.533 total time=   0.3s
[CV 2/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 0}, criterion=gini, max_depth=5, m

[CV 4/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 0}, criterion=gini, max_depth=7, max_features=auto, n_estimators=200;, score=0.509 total time=   0.4s
[CV 5/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 0}, criterion=gini, max_depth=7, max_features=auto, n_estimators=200;, score=0.472 total time=   0.4s
[CV 1/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 0}, criterion=gini, max_depth=7, max_features=auto, n_estimators=500;, score=0.533 total time=   1.1s
[CV 2/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 0}, criterion=gini, max_depth=7, max_features=auto, n_estimators=500;, score=0.491 total time=   1.1s
[CV 3/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 0}, criterion=gini, max_depth=7, max_features=auto, n_estimators=500;, score=0.513 total time=   1.1s
[CV 4/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 0}, criterion=gini, max_depth=7, m

[CV 1/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 0}, criterion=gini, max_depth=8, max_features=log2, n_estimators=200;, score=0.538 total time=   0.5s
[CV 2/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 0}, criterion=gini, max_depth=8, max_features=log2, n_estimators=200;, score=0.486 total time=   0.4s
[CV 3/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 0}, criterion=gini, max_depth=8, max_features=log2, n_estimators=200;, score=0.505 total time=   0.4s
[CV 4/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 0}, criterion=gini, max_depth=8, max_features=log2, n_estimators=200;, score=0.518 total time=   0.4s
[CV 5/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 0}, criterion=gini, max_depth=8, max_features=log2, n_estimators=200;, score=0.475 total time=   0.4s
[CV 1/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 0}, criterion=gini, max_depth=8, m

[CV 2/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 0}, criterion=entropy, max_depth=5, max_features=auto, n_estimators=500;, score=0.487 total time=   1.1s
[CV 3/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 0}, criterion=entropy, max_depth=5, max_features=auto, n_estimators=500;, score=0.500 total time=   1.1s
[CV 4/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 0}, criterion=entropy, max_depth=5, max_features=auto, n_estimators=500;, score=0.513 total time=   1.1s
[CV 5/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 0}, criterion=entropy, max_depth=5, max_features=auto, n_estimators=500;, score=0.485 total time=   1.1s
[CV 1/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 0}, criterion=entropy, max_depth=5, max_features=sqrt, n_estimators=200;, score=0.532 total time=   0.4s
[CV 2/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 0}, criterion=entro

...

[CV 3/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 5}, criterion=gini, max_depth=6, max_features=sqrt, n_estimators=500;, score=0.326 total time=   1.2s
[CV 4/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 5}, criterion=gini, max_depth=6, max_features=sqrt, n_estimators=500;, score=0.323 total time=   1.2s
[CV 5/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 5}, criterion=gini, max_depth=6, max_features=sqrt, n_estimators=500;, score=0.304 total time=   1.5s
[CV 1/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 5}, criterion=gini, max_depth=6, max_features=log2, n_estimators=200;, score=0.313 total time=   0.5s
[CV 2/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 5}, criterion=gini, max_depth=6, max_features=log2, n_estimators=200;, score=0.305 total time=   0.4s
[CV 3/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 5}, criterion=gini, max_depth=6, m

[CV 5/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 5}, criterion=gini, max_depth=8, max_features=auto, n_estimators=200;, score=0.330 total time=   0.5s
[CV 1/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 5}, criterion=gini, max_depth=8, max_features=auto, n_estimators=500;, score=0.339 total time=   1.4s
[CV 2/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 5}, criterion=gini, max_depth=8, max_features=auto, n_estimators=500;, score=0.343 total time=   1.4s
[CV 3/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 5}, criterion=gini, max_depth=8, max_features=auto, n_estimators=500;, score=0.380 total time=   1.5s
[CV 4/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 5}, criterion=gini, max_depth=8, max_features=auto, n_estimators=500;, score=0.336 total time=   1.4s
[CV 5/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 5}, criterion=gini, max_depth=8, m

[CV 1/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 5}, criterion=entropy, max_depth=4, max_features=log2, n_estimators=200;, score=0.296 total time=   0.4s
[CV 2/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 5}, criterion=entropy, max_depth=4, max_features=log2, n_estimators=200;, score=0.271 total time=   0.4s
[CV 3/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 5}, criterion=entropy, max_depth=4, max_features=log2, n_estimators=200;, score=0.285 total time=   0.4s
[CV 4/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 5}, criterion=entropy, max_depth=4, max_features=log2, n_estimators=200;, score=0.285 total time=   0.4s
[CV 5/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 5}, criterion=entropy, max_depth=4, max_features=log2, n_estimators=200;, score=0.276 total time=   0.4s
[CV 1/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 5}, criterion=entro

[CV 2/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 5}, criterion=entropy, max_depth=6, max_features=auto, n_estimators=500;, score=0.309 total time=   1.7s
[CV 3/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 5}, criterion=entropy, max_depth=6, max_features=auto, n_estimators=500;, score=0.331 total time=   1.7s
[CV 4/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 5}, criterion=entropy, max_depth=6, max_features=auto, n_estimators=500;, score=0.317 total time=   1.6s
[CV 5/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 5}, criterion=entropy, max_depth=6, max_features=auto, n_estimators=500;, score=0.307 total time=   1.5s
[CV 1/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 5}, criterion=entropy, max_depth=6, max_features=sqrt, n_estimators=200;, score=0.313 total time=   0.6s
[CV 2/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 5}, criterion=entro

[CV 3/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 5}, criterion=entropy, max_depth=7, max_features=log2, n_estimators=200;, score=0.367 total time=   0.8s
[CV 4/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 5}, criterion=entropy, max_depth=7, max_features=log2, n_estimators=200;, score=0.326 total time=   0.7s
[CV 5/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 5}, criterion=entropy, max_depth=7, max_features=log2, n_estimators=200;, score=0.315 total time=   0.7s
[CV 1/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 5}, criterion=entropy, max_depth=7, max_features=log2, n_estimators=500;, score=0.321 total time=   1.9s
[CV 2/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 5}, criterion=entropy, max_depth=7, max_features=log2, n_estimators=500;, score=0.328 total time=   1.9s
[CV 3/5] END class_weight={2: 0.8997150997150997, 1: 0.8997150997150997, 0: 5}, criterion=entro

In [23]:
accuracy_score(grid.best_estimator_.predict(X_test), y_test)

0.5537974683544303

In [24]:
grid.best_params_

{'class_weight': {2: 0.8997150997150997, 1: 0.8997150997150997, 0: 1},
 'criterion': 'gini',
 'max_depth': 5,
 'max_features': 'auto',
 'n_estimators': 200}